# Redis Sets Tutorial

## Setup

In [1]:
%%capture
%pip install redis rich tqdm 

In [2]:
import redis
from rich.pretty import pprint

In [3]:
# creating cluster
# make sure jupyter server is connected to redis network
# `docker network connect redis_default jupyter-jupyter-1`
r = redis.RedisCluster(host='master', port=6379)

# Redis Sets

Redis sets are similar to lists, except they are unordered, and each element must be unique. You can use sets to:

- Track unique items (e.g., track all unique IP addresses accessing a given blog post).
- Represent relations (e.g., the set of all users with a specified role).
- Perform common set operations such as intersection, union, and difference.

Here are some important set commands:

- `SADD`
- `SREM`
- `SISMEMBER`
- `SMEMBERS`
- `SUNION`

Use `SADD` to create and update a set. Each `SADD` command will return the number of added members. If you try to add a member that is already in the set, `0` is returned.

In [7]:
# Create a set
result1 = r.sadd("{bike:1}:addons", "whitewall tires")
print(f"Added 'whitewall tires': {result1}")

result2 = r.sadd("{bike:1}:addons", "bell", "reflectors")
print(f"Added 'bell' and 'reflectors': {result2}")

result3 = r.sadd("{bike:1}:addons", "bell")
print(f"Tried to add 'bell' again: {result3}")

Added 'whitewall tires': 0
Added 'bell' and 'reflectors': 1
Tried to add 'bell' again: 0


Notice that the `SADD` command is variadic.

`SREM` is used to remove members of a set. It returns `1` if the member is in the set, or `0` if it is not.

In [8]:
# Remove set members
result1 = r.srem("{bike:1}:addons", "bell")
print(f"Removed 'bell': {result1}")

result2 = r.srem("{bike:1}:addons", "sissy bar")
print(f"Tried to remove 'sissy bar' (not in set): {result2}")

Removed 'bell': 1
Tried to remove 'sissy bar' (not in set): 0


The `SISMEMBER` command is used to test set membership. It returns `1` if the member is present, and `0` otherwise.

In [9]:
# Test set membership
result1 = r.sismember("{bike:1}:addons", "reflectors")
print(f"Is 'reflectors' in the set? {bool(result1)}")

result2 = r.sismember("{bike:1}:addons", "sissy bar")
print(f"Is 'sissy bar' in the set? {bool(result2)}")

Is 'reflectors' in the set? True
Is 'sissy bar' in the set? False


Use `SMEMBERS` to return all the members of a set.

In [10]:
# Get set members
members = r.smembers("{bike:1}:addons")
print("Set members:")
pprint(members)

Set members:


{b'whitewall tires', b'reflectors'}

`SUNION` combines two or more sets and returns all their elements.

In [11]:
# SUNION usage
r.sadd("{bike:1}:special_addons", "sparkle coat finish", "banana seat")
print("Special addons:")
pprint(r.smembers("{bike:1}:special_addons"))

union_result = r.sunion("{bike:1}:addons", "{bike:1}:special_addons")
print("\nUnion of both sets:")
pprint(union_result)

Special addons:


{b'banana seat', b'sparkle coat finish'}


Union of both sets:


{b'banana seat', b'whitewall tires', b'reflectors', b'sparkle coat finish'}

Sets have two ways to return one or more random members of a set:

- `SPOP`
- `SRANDMEMBER`

Each command takes a key and, optionally, a count as arguments. `SPOP` removes and returns a random member, whereas `SRANDMEMBER` just returns the randomly selected members without removing them.

In [12]:
# SPOP/SRANDMEMBER usage
print("Current members of bike:1:addons:")
pprint(r.smembers("{bike:1}:addons"))

print("\nRandom member (without removing):")
pprint(r.srandmember("{bike:1}:addons"))

print("\nRandom member (with removal):")
popped = r.spop("{bike:1}:addons")
pprint(popped)

print("\nRemaining members after SPOP:")
pprint(r.smembers("{bike:1}:addons"))

Current members of bike:1:addons:


{b'whitewall tires', b'reflectors'}


Random member (without removing):


b'reflectors'


Random member (with removal):


b'whitewall tires'


Remaining members after SPOP:


{b'reflectors'}

### Resources

- Set type [reference page](https://redis.io/docs/data-types/sets?utm_source=redisinsight&utm_medium=main&utm_campaign=tutorials).
- Entire list of [Redis set commands](https://redis.io/commands/?group=set&utm_source=redisinsight&utm_medium=main&utm_campaign=tutorials).
- Check out [Get started with Redis](https://university.redis.io/learningpath/14q8m6gilfwltm?utm_source=redisinsight&utm_medium=main&utm_campaign=tutorials) learning path on Redis University for an introduction to working with all core data structures in Redis.